## Testing locally pytorch trasnformer functionalities

General performance runtime generated:
- CPU local: too much!
- M1 Pro (with native python installation , Pytorch nightly etc.): 250s per epoch
- Cuda GPU on Colab: 47s

Even if Colab is 5x time faster I'd go with M1 Pro for prototyping and using Colab/Cloud/on-prem only when needed:
- PyTorch MPS is still under big improvements. TF demonstrated to be faster on M1 Pro vs Colab
- Easier to prototype end-to-end code
- Faster on I/O performances and on inference
- Scalable on cloud/on-prem easily vs Colab
It's important to see if any new updates will be for PyTorch and PyTorch lightning (since not fully supported yet), that can give greater scalability vs TF.

In [7]:
#TODO
# dockerize also for CUDA (https://towardsdatascience.com/how-to-properly-use-the-gpu-within-a-docker-container-4c699c78c6d1), optimize code (https://huggingface.co/docs/transformers/v4.18.0/en/performance)
# train on wikitext 103 on Cloud with docker
# fine tune on twitter dataset language model locally
# fine tune on twitter dataset sentiment classification locally
# test against BERT pre-trained and choose which to use
# checkpointing the best 2nd stage language model and only the encoder (The model not including the task-specific final layer(s)) -- See Fastai to take inspiration
# dockerise fine tuning
# fix requirements.txt  --> it's a mess with pytorch nightly installed (via pip3 that I enabled looking at dev apple forum for installing python nativaly with miniforge on M1, see docs, adding torchtext and torchdata)
# map the design
# add ZenML and build the final ML pipeline for training
# dockerise the overall pipeline, make it multi-arch to enable CUDA GPUs (and test them). Add Poetry (or conda) and package dependency mnager
# log test metrics at each run of the pipeline + create custom metrics for dataset quality
# final refinement: look at TODOs in the code, cleanup useless code, create README.md
# Check if pytorch lighting applicable (https://towardsdatascience.com/from-pytorch-to-pytorch-lightning-a-gentle-introduction-b371b7caaf09) etc.
# look for updates on Pytorch for MPS on MacM1 and for updates of MacOs arch images on Docker (for using PyTorch MPS on docker)
# Look for Pytorch updates for Linux aarch64 in pip or conda (https://anaconda.org/pytorch/)
# create and dockerise deployment for SentimentClassifier (TorchServe)

In [8]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset


class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        src = self.encoder(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, src_mask)
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Generates an upper-triangular matrix of -inf, with zeros on diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [10]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")
else:
    print("MPS is available.")


MPS is available.


In [11]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Converts raw text into a flat Tensor.
    Collates raw text into a unique vector"""
    # use vocab created and tokenizer given
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) #consider only the tokenized tensors that have at least one element

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divides the data into bsz separate sequences, removing extra elements
    that wouldn't cleanly fit (not padding)

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
        with sequence lenght = N // bsz
    """
    seq_len = data.size(0) // bsz
    data = data[:seq_len * bsz]
    data = data.view(bsz, seq_len).t().contiguous()
    return data.to(device)

In [12]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])


/Users/andreatamburri/miniforge3/envs/m1-env/lib/python3.10/site-packages/torch/utils/data/graph.py:102: FutureWarning: `only_datapipe` is deprecated from `traverse` function and will be removed after 1.13.
  warnings.warn(msg, FutureWarning)


In [13]:
vocab.lookup_token(3869)

'chronicles'

In [14]:
# lenght of the iterator
sum(1 for _ in train_iter)

36718

In [15]:
# print some samples
for i,item in enumerate(train_iter):
    if i <= 5:
        print(i, f' len is {len(item)}', '\n', item)
    else:
        break

0  len is 2 
  

1  len is 30 
  = Valkyria Chronicles III = 

2  len is 2 
  

3  len is 694 
  Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . <unk> the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 

4  len is 520 
  The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multip

In [16]:
# train_iter was "consumed" by the process of building the vocab,
# so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

#device = torch.device('cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')  #500s for an epoch, at least 10/20x faster


In [17]:
device

device(type='mps')

In [30]:
type(device)

torch.device

In [18]:
#all the words of the iterator
train_data

tensor([   9, 3849, 3869,  ..., 2442, 4810,    3])

In [19]:
#batchify
batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [20]:
train_data

tensor([[    9,    59,   564,  ..., 11652,  2435,     1],
        [ 3849,    12,   300,  ...,    47,    30,  1990],
        [ 3869,   315,    19,  ...,    97,  7720,     4],
        ...,
        [  587,  4011,    59,  ...,     1,  1439, 12313],
        [ 4987,    29,     4,  ...,  3165, 17106,  2060],
        [    6,     8,     1,  ...,    62,    18,     2]], device='mps:0')

In [21]:
bptt = 35  # backpropagation through time, real sequence lenght fed into the transformer_encoder
def get_batch(source: Tensor, i: int, bptt) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

In [22]:
# mask at encoder level since is a language model, so we have to mask future positions
# at the qi*Kt level, where Kt for i from 0 to btt
# when i=0 (first word), we cannot take into account non of Kt with t>i
src_mask = generate_square_subsequent_mask(bptt).to(device)
src_mask

/Users/andreatamburri/miniforge3/envs/m1-env/lib/python3.10/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::masked_select' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0')

In [23]:
data, target = get_batch(train_data, 0)
# first minibatch
data

tensor([[    9,    59,   564,   223,   443, 13627,     2,   539,  2872,  2464,
             0,   313,  4513,     1,     5,    47,    66, 11652,  2435,     1],
        [ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
             2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990],
        [ 3869,   315,    19,    29,   939,     2,    10,  2139,  4916, 16615,
           235,     3,    13,     7,    24,    17, 13737,    97,  7720,     4],
        [  881,    67,   807,  5402,     6,    38, 28188,    25,     2,    77,
             7,  2394,    17,   516,    14, 16403,  3714,  4618,    12,  1108],
        [    9,   196,  6041,   190,   218, 11776,    17,     1,  1200,     2,
             0,    10,   591,    40,  6004,     2,    50,     3,  3131,  3781],
        [20000,    21,  6406,    14,     3,    10,     8,   114,    24,  2294,
          1684,  7156,  1681,   191,   928,  5879,    51,     9,  1147,     5],
        [   83,     8,     2,  1019,  1817,   

In [24]:
data, target = get_batch(train_data, 1)
""" how to read it?
Counterintuitive because the sequence of words is not orizontal.
Sequence is the first dimension, so you have to read vertically.
 With 9 you can predict 3849, with 9 & 3849 you can predict 3869 and so on
 until being able to predict the 35th word considering the first 34 words"""

data

tensor([[ 3849,    12,   300,  6302,  3989,  1930, 10559,   451,     4,     7,
             2,  1511, 10115,   942,  2439,   572,     1,    47,    30,  1990],
        [ 3869,   315,    19,    29,   939,     2,    10,  2139,  4916, 16615,
           235,     3,    13,     7,    24,    17, 13737,    97,  7720,     4],
        [  881,    67,   807,  5402,     6,    38, 28188,    25,     2,    77,
             7,  2394,    17,   516,    14, 16403,  3714,  4618,    12,  1108],
        [    9,   196,  6041,   190,   218, 11776,    17,     1,  1200,     2,
             0,    10,   591,    40,  6004,     2,    50,     3,  3131,  3781],
        [20000,    21,  6406,    14,     3,    10,     8,   114,    24,  2294,
          1684,  7156,  1681,   191,   928,  5879,    51,     9,  1147,     5],
        [   83,     8,     2,  1019,  1817,    39,   971,  2581,     1, 12522,
             2,    19,     7,    26,     2,     5,  1104,     9,  2995,   986],
        [ 3849,   236,    52,  5067,    50,   

In [25]:
target

tensor([ 3869,   315,    19,    29,   939,     2,    10,  2139,  4916, 16615,
          235,     3,    13,     7,    24,    17, 13737,    97,  7720,     4,
          881,    67,   807,  5402,     6,    38, 28188,    25,     2,    77,
            7,  2394,    17,   516,    14, 16403,  3714,  4618,    12,  1108,
            9,   196,  6041,   190,   218, 11776,    17,     1,  1200,     2,
            0,    10,   591,    40,  6004,     2,    50,     3,  3131,  3781,
        20000,    21,  6406,    14,     3,    10,     8,   114,    24,  2294,
         1684,  7156,  1681,   191,   928,  5879,    51,     9,  1147,     5,
           83,     8,     2,  1019,  1817,    39,   971,  2581,     1, 12522,
            2,    19,     7,    26,     2,     5,  1104,     9,  2995,   986,
         3849,   236,    52,  5067,    50,     3,   818,   131,  1314,    13,
           10,  5136,   481,     1,    53,  9503,     7,  9584, 10187,    18,
           88,   152,  3054,    29,   214,   130,     2,     5, 

In [26]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [27]:
for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        print(seq_len)

35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
35
3

In [28]:
import copy
import time

criterion = nn.CrossEntropyLoss()
lr = 5.0  # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

def train(model: nn.Module) -> None:
    model.train()  # turn on train mode
    total_loss = 0.
    log_interval = 200
    start_time = time.time()
    src_mask = generate_square_subsequent_mask(bptt).to(device)

    num_batches = len(train_data) // bptt
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)
        seq_len = data.size(0)
        if seq_len != bptt:  # only on last batch that can be less then bptt. During batchify instead we delete remaining words, no padding
            src_mask = src_mask[:seq_len, :seq_len]
        output = model(data, src_mask)
        loss = criterion(output.view(-1, ntokens), targets)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval
            cur_loss = total_loss / log_interval
            ppl = math.exp(cur_loss)
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # turn on evaluation mode
    total_loss = 0.
    src_mask = generate_square_subsequent_mask(bptt).to(device)
    with torch.no_grad():
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i)
            seq_len = data.size(0)
            if seq_len != bptt:
                src_mask = src_mask[:seq_len, :seq_len]
            output = model(data, src_mask)
            output_flat = output.view(-1, ntokens)
            total_loss += seq_len * criterion(output_flat, targets).item()
    return total_loss / (len(eval_data) - 1)

In [29]:
best_val_loss = float('inf')
epochs = 3
best_model = None

for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(model)
    val_loss = evaluate(model, val_data)
    val_ppl = math.exp(val_loss)
    elapsed = time.time() - epoch_start_time
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = copy.deepcopy(model)

    scheduler.step()

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 110.78 | loss  8.43 | ppl  4591.68
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 89.79 | loss  7.53 | ppl  1854.13
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 85.35 | loss  7.21 | ppl  1355.95
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 84.35 | loss  7.11 | ppl  1222.06
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 84.08 | loss  7.08 | ppl  1187.29
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 86.50 | loss  7.04 | ppl  1145.09
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 85.35 | loss  7.00 | ppl  1099.36
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 87.97 | loss  7.00 | ppl  1100.22
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 87.82 | loss  6.99 | ppl  1087.22
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 85.86 | loss  6.99 | ppl  1080.88
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 85.11 | loss  6.95 | ppl  1047.55
| epoch   1 |  2400/